In [3]:
import numpy as np
numpyData = np.zeros((1,10,10,3), dtype=np.float32)
import tensorflow as tf
tf.convert_to_tensor(numpyData)

<tf.Tensor 'Const:0' shape=(1, 10, 10, 3) dtype=float32>

In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tf.constant([1,2,3]).eval(session=sess)

array([1, 2, 3], dtype=int32)

In [9]:
[1,2,3,5][1:3+1]

[2, 3, 5]

In [13]:
np.shape(np.reshape(np.array([1,2,3]), [-1]))

(3,)

In [14]:
np.shape(np.array([1,2,3]))

(3,)

In [16]:
from numpy import array
from scipy.sparse import coo_matrix
row  = array([0,0,1,3,1,0,0])
col  = array([0,2,1,3,1,0,0])
data = array([1,1,1,1,1,1,1])
A = coo_matrix( (data,(row,col)), shape=(4,4))
A.todense()

matrix([[3, 0, 1, 0],
        [0, 2, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1]])

In [19]:
[1,2,3,3,4,5,6,7,4][-2:]

[7, 4]

In [21]:
[1,2,3,3,4,5,6,7,4][1:-3]

[2, 3, 3, 4, 5]

In [23]:
np.matrix([[1,2],[3,4]]).todense()

AttributeError: 'matrix' object has no attribute 'todense'

In [78]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)
with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1: [[7.,2.,1.],[1.,1.,1.]], input2: [[3.,1.,1.], [1.,1.,1.]]}))

[[ 21.   2.   1.]
 [  1.   1.   1.]]


In [33]:
def init_var_map(init_vars, init_path=None):
    var_map = {}
    for var_name, var_shape, init_method, dtype in init_vars:
        if init_method == 'xavier':
            maxval = np.sqrt(6. / np.sum(var_shape))
            minval = -maxval
            var_map[var_name] = tf.Variable(tf.random_uniform(var_shape, minval=minval, maxval=maxval, dtype=dtype),
                                            name=var_name, dtype=dtype)
    return var_map

In [72]:
field_sizes = [3,4,5]
embed_size = 2
init_vars = []
num_inputs = len(field_sizes)
for i in range(num_inputs):
    for j in range(num_inputs):
        init_vars.append(('embed_%d_%d' % (i, j), [field_sizes[i], embed_size], 'xavier', tf.float32))

X = [tf.sparse_placeholder(tf.float32) for i in range(num_inputs)]
y = tf.placeholder(tf.float32)

svars = init_var_map(init_vars)
print(svars)
w0 = []
for i in range(num_inputs):
    w_temp = []
    for j in range(num_inputs):
        w_temp.append(svars['embed_%d_%d' % (i, j)])
    w0.append(w_temp)
#self.vars['embed_%d_%d' % i % j] for i in range(num_inputs) for j in range(num_inputs)]
#field_dense_fieldi_fieldj = [[tf.sparse_tensor_dense_matmul(self.X[i], w0[i * num_inputs + j]) for i in range(num_inputs)] for j in range(num_inputs)]
dense_fieldij = []
for i in range(num_inputs):
    dense_temp = []
    for j in range(num_inputs):
        dense_temp.append(tf.sparse_tensor_dense_matmul(X[i], w0[i][j]))
    dense_fieldij.append(dense_temp)

product_results = []
for i in range(num_inputs - 1):
    for j in range(i, num_inputs - i):
        product_results.append(tf.matmul(dense_fieldij[i][j], dense_fieldij[j][i]))
l = tf.reduce_sum(product_results)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=l, labels=y))
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

from scipy.sparse import coo_matrix

field1 = coo_matrix((np.array([1.,1.]), (np.array([0,1]), np.array([0,1]))), shape = (2,3)).tocoo()
field2 = coo_matrix((np.array([1.,1.]), (np.array([0,1]), np.array([0,1]))), shape = (2,4)).tocoo()
field3 = coo_matrix((np.array([1.,1.]), (np.array([0,1]), np.array([0,1]))), shape = (2,5)).tocoo()
print(field1)
data1 = np.vstack((field1.row, field1.col)).transpose(), field1.data, field1.shape
data2 = np.vstack((field2.row, field2.col)).transpose(), field2.data, field2.shape
data3 = np.vstack((field3.row, field3.col)).transpose(), field3.data, field3.shape
data= [data1, data2, data3]

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    feed_dict1 = {}
    for i in range(len(X)):
        feed_dict1[X[i]] = data[i]
    feed_dict1[y] = np.array([0,1])
    for _ in range(10):
        sess.run(optimizer, feed_dict=feed_dict1)
        if _ % 2 == 0:
            print(sess.run(loss, feed_dict=feed_dict1))

{'embed_0_2': <tf.Variable 'embed_0_2_24:0' shape=(3, 2) dtype=float32_ref>, 'embed_0_0': <tf.Variable 'embed_0_0_24:0' shape=(3, 2) dtype=float32_ref>, 'embed_0_1': <tf.Variable 'embed_0_1_24:0' shape=(3, 2) dtype=float32_ref>, 'embed_2_0': <tf.Variable 'embed_2_0_24:0' shape=(5, 2) dtype=float32_ref>, 'embed_2_1': <tf.Variable 'embed_2_1_24:0' shape=(5, 2) dtype=float32_ref>, 'embed_2_2': <tf.Variable 'embed_2_2_24:0' shape=(5, 2) dtype=float32_ref>, 'embed_1_2': <tf.Variable 'embed_1_2_24:0' shape=(4, 2) dtype=float32_ref>, 'embed_1_1': <tf.Variable 'embed_1_1_24:0' shape=(4, 2) dtype=float32_ref>, 'embed_1_0': <tf.Variable 'embed_1_0_24:0' shape=(4, 2) dtype=float32_ref>}
  (0, 0)	1.0
  (1, 1)	1.0
0.694724
0.69315
0.693147
0.693147
0.693147


In [42]:
print('adfasfd%d%d' % (1))

TypeError: not enough arguments for format string

In [75]:
[1 for x in range(3,4)]

[1]

In [146]:
input1 = tf.placeholder(tf.float32)
output = tf.reduce_sum(input1,0)
with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1: [[7.,2.,1.],[1.,1.,1.]]}))
    print(tf.shape(output))
    print(tf.shape(input1))

[ 8.  3.  2.]
Tensor("Shape_9:0", shape=(?,), dtype=int32)
Tensor("Shape_10:0", shape=(?,), dtype=int32)


In [99]:
strqueue = tf.train.string_input_producer(
    tf.train.match_filenames_once(
        '/Users/laisijia/git/make-ipinyou-data/2259/train.txt*'))

In [100]:
_,a = tf.TFRecordReader().read(strqueue)

In [101]:
a

<tf.Tensor 'ReaderReadV2_2:1' shape=() dtype=string>

In [102]:
batch_serialized_example = tf.train.shuffle_batch([a],
    batch_size=1024, num_threads=3, capacity=50000,
    min_after_dequeue=10000,
    allow_smaller_final_batch=True
  )

In [103]:
batch_serialized_example

<tf.Tensor 'shuffle_batch_1:0' shape=(?,) dtype=string>

In [97]:
features = tf.parse_example(batch_serialized_example, features={
    'label': tf.FixedLenFeature([1], tf.float32),
    'instance_id': tf.FixedLenFeature([1], tf.int64),
    'feature_id': tf.VarLenFeature(tf.int64),
    'value': tf.VarLenFeature(tf.float32),
    'tree_feature_id': tf.VarLenFeature(tf.int64),
    'tree_value': tf.VarLenFeature(tf.float32),
  })

In [98]:
features

{'feature_id': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1154e7950>,
 'instance_id': <tf.Tensor 'ParseExample/ParseExample:12' shape=(?, 1) dtype=int64>,
 'label': <tf.Tensor 'ParseExample/ParseExample:13' shape=(?, 1) dtype=float32>,
 'tree_feature_id': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1154e77d0>,
 'tree_value': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1154e76d0>,
 'value': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1154e7450>}

In [125]:
a = tf.constant([[1,2,3],[4,5,6],[7,8,9]])
z=tf.segment_prod(a,[2,2,2])
with tf.Session() as sess:
    print(sess.run(z))

[[  1   1   1]
 [  1   1   1]
 [ 28  80 162]]


In [139]:
np.array([[1,1,1],[2,2,2]])[0,:]

array([1, 1, 1])

In [141]:
a,b = tf.unique([0,0,1,1,1])
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))

[0 1]
[0 0 1 1 1]


In [143]:
np.array([1,1,1]).indices

AttributeError: 'numpy.ndarray' object has no attribute 'indices'

In [160]:
a = tf.reshape([0],shape=[0])

ValueError: Cannot reshape a tensor with 1 elements to shape [0] (0 elements) for 'Reshape_6' (op: 'Reshape') with input shapes: [1], [1] and with input tensors computed as partial shapes: input[1] = [0].

In [159]:
a

<tf.Tensor 'Reshape_5:0' shape=() dtype=int32>

In [154]:
tf.shape(a)

<tf.Tensor 'Shape_14:0' shape=(3,) dtype=int32>

In [155]:
a

<tf.Tensor 'Reshape_3:0' shape=(?, 0, 1) dtype=int32>

In [156]:
a[0]

<tf.Tensor 'strided_slice:0' shape=(0, 1) dtype=int32>

In [162]:
tf.reshape([1,1,1], (-1, 1))

<tf.Tensor 'Reshape_8:0' shape=(3, 1) dtype=int32>

In [166]:
tf.reshape([1,1,1], (-1,0))

<tf.Tensor 'Reshape_11:0' shape=(?, 0) dtype=int32>

In [164]:
tf.shape([1,1,1])

<tf.Tensor 'Shape_15:0' shape=(1,) dtype=int32>

In [168]:
a = tf.string_split(["1 1:0 2:0"], " ")

In [169]:
a.values

<tf.Tensor 'StringSplit_1:1' shape=(?,) dtype=string>

In [170]:
[str(i) + ":" + str(x) for i,x in enumerate([1,1,1,1])]

['0:1', '1:1', '2:1', '3:1']